# file_bridge — SCP ファイル転送ツール

このノートブックは `main.py` と同等の操作を Jupyter Notebook 上で実行します。

- **ダウンロード**: リモートサーバーからファイルを取得
- **アップロード**: ローカルファイルをリモートサーバーへ送信
- **チェックサム検証**: SHA-256 による転送整合性チェック
- **転送サマリー**: 完了後に HTML テーブルで結果を表示

---
**実行前に**: このノートブックはプロジェクトルート（`main.ipynb` があるディレクトリ）から起動してください。

## 1. セットアップ

In [2]:
import logging
import sys
from pathlib import Path

# ロギング設定
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

# src パッケージをインポート
from src.client import SCPClient
from src.config import ConfigLoader

print("インポート完了")

インポート完了

## 2. 接続設定

### 方法 A: config.yaml のプロファイルを使用（推奨）

プロジェクトルートの `config.yaml` に定義されたプロファイルから接続情報を読み込みます。

In [3]:
# config.yaml からプロファイルを読み込む
loader = ConfigLoader("config.yaml")

print(f"利用可能なプロファイル: {loader.available_profiles}")
print(f"デフォルトプロファイル: {loader.default_profile}")

# 使用するプロファイルを指定（None でデフォルト使用）
PROFILE_NAME = None  # 例: "jupyter", "production"

profile = loader.get_profile(PROFILE_NAME)
print(f"\n接続先: {profile.user}@{profile.host}:{profile.port}")
print(f"リモートベース: {profile.remote_base}")
print(f"ローカルベース: {profile.local_base}")
print(f"チェックサム: {profile.checksum}")

2026-02-23 15:27:25 - INFO - 設定ファイルを読み込みました: config.yaml (2 プロファイル)

利用可能なプロファイル: ['default', 'jupyter']
デフォルトプロファイル: jupyter

接続先: sysope@192.168.3.61:22
リモートベース: /home/jupyter_projects/test/
ローカルベース: ./downloads/
チェックサム: True

In [4]:
# プロファイルから SCPClient を生成
client = SCPClient(
    host=profile.host,
    user=profile.user,
    password=profile.password,  # config.yaml に記載がない場合は文字列で直接指定
    port=profile.port,
    log_file=profile.log,
    use_checksum=profile.checksum,
)
print(f"SCPClient 作成完了: {client.user}@{client.host}")

SCPClient 作成完了: sysope@192.168.3.61

### 方法 B: 直接指定

接続情報をコードで直接指定する場合は以下のセルを実行してください（方法 A の代わりに使用）。

In [ ]:
# # 接続情報を直接指定する場合（方法 A の代わり）
# import getpass
#
# client = SCPClient(
#     host="192.168.3.61",
#     user="sysope",
#     password=getpass.getpass("パスワード: "),  # 対話入力
#     port=22,
#     log_file="./logs/transfer.log",
#     use_checksum=True,
# )

## 3. ダウンロード

`remote` にはワイルドカード（`*.csv`、`data_*` など）が使用できます。

In [5]:
# ダウンロードするリモートパス（ワイルドカード可）
REMOTE_PATH = "/home/jupyter_projects/test/test-file*.txt"

# ローカル保存先ディレクトリ
LOCAL_DIR = "./downloads/"

# ダウンロード実行
download_results = client.download(
    remote=REMOTE_PATH,
    local=LOCAL_DIR,
)
# ↑ 完了後に HTML テーブルのサマリーが自動表示されます

'[3/3] downloading test-file3.txt...'

2026-02-23 15:27:35 - INFO - Connected (version 2.0, client OpenSSH_9.6p1)2026-02-23 15:27:35 - INFO - Authentication (password) successful!

ファイル,状態,エラー
/home/jupyter_projects/test/test-file1.txt,✓ 成功,
/home/jupyter_projects/test/test-file2.txt,✓ 成功,
/home/jupyter_projects/test/test-file3.txt,✓ 成功,


In [6]:
# ダウンロード結果の確認
print("=== ダウンロード結果 ===")
for r in download_results:
    status = "✓" if r["success"] else "✗"
    print(f"{status} {r['remote']} -> {r['local']}")
    if r["error"]:
        print(f"  エラー: {r['error']}")

# ダウンロードされたファイルを一覧表示
print(f"\n{LOCAL_DIR} の内容:")
for f in sorted(Path(LOCAL_DIR).glob("*")):
    print(f"  {f.name} ({f.stat().st_size:,} bytes)")

=== ダウンロード結果 ===
✓ /home/jupyter_projects/test/test-file1.txt -> downloads/test-file1.txt
✓ /home/jupyter_projects/test/test-file2.txt -> downloads/test-file2.txt
✓ /home/jupyter_projects/test/test-file3.txt -> downloads/test-file3.txt

./downloads/ の内容:
  .ipynb_checkpoints (4,096 bytes)
  test-file1.txt (1,152 bytes)
  test-file2.txt (6,636 bytes)
  test-file3.txt (658 bytes)

## 4. アップロード

`local` にはワイルドカードが使用できます。リモートディレクトリは自動作成されます。

In [7]:
# アップロードするローカルファイル（ワイルドカード可）
LOCAL_UPLOAD_PATH = "./downloads/test-file*.txt"

# リモート保存先ディレクトリ
REMOTE_UPLOAD_DIR = "/home/jupyter_projects/test/uploaded/"

# アップロード実行
upload_results = client.upload(
    local=LOCAL_UPLOAD_PATH,
    remote=REMOTE_UPLOAD_DIR,
)
# ↑ 完了後に HTML テーブルのサマリーが自動表示されます

'[3/3] uploading test-file3.txt...'

2026-02-23 15:27:45 - INFO - Connected (version 2.0, client OpenSSH_9.6p1)2026-02-23 15:27:45 - INFO - Authentication (password) successful!

ファイル,状態,エラー
/home/jupyter_projects/test/uploaded/test-file1.txt,✓ 成功,
/home/jupyter_projects/test/uploaded/test-file2.txt,✓ 成功,
/home/jupyter_projects/test/uploaded/test-file3.txt,✓ 成功,


In [8]:
# アップロード結果の確認
print("=== アップロード結果 ===")
for r in upload_results:
    status = "✓" if r["success"] else "✗"
    print(f"{status} {r['local']} -> {r['remote']}")
    if r["error"]:
        print(f"  エラー: {r['error']}")

=== アップロード結果 ===
✓ downloads/test-file1.txt -> /home/jupyter_projects/test/uploaded/test-file1.txt
✓ downloads/test-file2.txt -> /home/jupyter_projects/test/uploaded/test-file2.txt
✓ downloads/test-file3.txt -> /home/jupyter_projects/test/uploaded/test-file3.txt

## 5. 転送ログの確認

In [9]:
# 転送ログを表示
log_file = Path(profile.log)

if log_file.exists():
    print(f"=== {log_file} ===")
    lines = log_file.read_text(encoding="utf-8").strip().splitlines()
    # 最新 10 行を表示
    for line in lines[-10:]:
        print(line)
    print(f"\n（全 {len(lines)} 件）")
else:
    print(f"ログファイルがまだ存在しません: {log_file}")

=== logs/transfer.log ===
2026-02-23 15:06:41 | DOWNLOAD | /home/jupyter_projects/test/test-file3.txt -> downloads/test-file3.txt | 658B | SUCCESS | SHA256: f7f4d6b1b5db630bd63a71f9aab4532c971863f2c07245a59617b5c186281d10
2026-02-23 15:07:24 | UPLOAD   | downloads/test-file1.txt -> /home/jupyter_projects/test/uploaded/test-file1.txt | 1.1KB | SUCCESS | SHA256: 9fa61f736513edcfefab02416cf289f80372451c9af4dbbd27d5362569c1afb1
2026-02-23 15:07:25 | UPLOAD   | downloads/test-file2.txt -> /home/jupyter_projects/test/uploaded/test-file2.txt | 6.5KB | SUCCESS | SHA256: 985b5865e57896014473b3ed1beb57feaf9234b712ad9d85583907c37e813cbe
2026-02-23 15:07:25 | UPLOAD   | downloads/test-file3.txt -> /home/jupyter_projects/test/uploaded/test-file3.txt | 658B | SUCCESS | SHA256: f7f4d6b1b5db630bd63a71f9aab4532c971863f2c07245a59617b5c186281d10
2026-02-23 15:27:34 | DOWNLOAD | /home/jupyter_projects/test/test-file1.txt -> downloads/test-file1.txt | 1.1KB | SUCCESS | SHA256: 9fa61f736513edcfefab02416cf28